Flores Lara Alberto
5BV1
Ingenieria en Inteligencia Artificial
10/06/24

In [10]:
# Importar las librerías necesarias
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus.reader import PlaintextCorpusReader
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from scipy import spatial

# Descarga de los recursos necesarios
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('stopwords')

# Definir las funciones del programa

# Convierte todo el texto a minúsculas
def minusculas(texto):
    return texto.lower()

#Elimina todos los caracteres especiales del texto
def caracteresesp(texto):
    return re.sub(r'[^a-zA-Z0-9\s]', '', texto)

# Quita los espacios extra del texto
def quitaresp(texto):
    return " ".join(texto.split())

#Elimina las palabras comunes stopwords
def stooopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [w for w in tokens if w.lower() not in stop_words]

#Tokeniza el texto
def tokenizar(texto):
    return word_tokenize(texto)

def aplicar_procesamiento(cuerpos):
    cuerpos_procesados = []
    for cuerpo in cuerpos:
        cuerpo_procesado = minusculas(cuerpo)
        cuerpo_procesado = caracteresesp(cuerpo_procesado)
        cuerpo_procesado = quitaresp(cuerpo_procesado)
        tokens = tokenizar(cuerpo_procesado)
        tokens = stooopwords(tokens)
        cuerpos_procesados.append(tokens)
    return cuerpos_procesados

# Limpiamos todos los caracteres que no esten en formato utf-8
def clean_text(text):
    return text.encode('utf-8', 'ignore').decode('utf-8')

# Función para etiquetar gramaticalmente el texto
def pos_tag_text(text):
    sentences = nltk.sent_tokenize(text)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    return tagged_sentences

# Función para encontrar la palabra más frecuente de una categoría gramatical dada
def most_frequent_word(tagged_text, pos_tag_prefix):
    words = [word for word, tag in tagged_text if tag.startswith(pos_tag_prefix)]
    fdist = nltk.FreqDist(words)
    return fdist.max()

# Función para calcular la similitud entre palabras usando synsets
def word_similarity(word, pos_tag, metric='wup_similarity'):
    synsets = wn.synsets(word, pos=pos_tag)
    if not synsets:
        return []
    
    similarities = []
    for synset in synsets:
        for other_synset in synsets:
            if synset != other_synset:
                if metric == 'wup_similarity':
                    similarity = synset.wup_similarity(other_synset)
                elif metric == 'path_similarity':
                    similarity = synset.path_similarity(other_synset)
                else:
                    similarity = 0
                if similarity is not None:
                    similarities.append((other_synset, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:5]

# Función para convertir etiquetas a las utilizadas por WordNet
def convert_tag(tag):
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

# Función para convertir documento a synsets
def doc_to_synsets(doc):
    tokens = nltk.word_tokenize(doc)
    pos = nltk.pos_tag(tokens)
    wntag = [convert_tag(tag) for tag in [tag[1] for tag in pos]]
    synsets = [wn.synsets(token, tag)[0] for token, tag in zip(tokens, wntag) if wn.synsets(token, tag)]
    return synsets

# Función para calcular la puntuación de similitud
def similarity_score(s1, s2):
    scores = []
    for synset in s1:
        best_score = max([synset.path_similarity(ss) for ss in s2 if synset.path_similarity(ss)] or [0])
        if best_score:
            scores.append(best_score)
    return sum(scores) / len(scores) if scores else 0

# Función para calcular la similitud entre dos documentos
def document_path_similarity(doc1, doc2):
    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)
    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

# Función para cargar las incrustaciones de GloVe
def load_glove_embeddings(glove_file):
    embeddings_dict = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], 'float32')
            embeddings_dict[word] = vectors
    return embeddings_dict

# Función para encontrar las palabras más similares usando GloVe
def find_closest_embeddings(embedding, embeddings_dict, top_n=5):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.cosine(embeddings_dict[word], embedding))[:top_n]

# Función para obtener las top palabras representativas de un texto usando BERT
def obtener_top_palabras(texto, model, tokenizer, top_n=5):
    # Tokenizamos el texto
    tokens = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
    
    # Pasamos el input por BERT para obtener los embeddings
    with torch.no_grad():
        output = model(**tokens)
    word_embeddings = output.last_hidden_state.squeeze(0)
    
    # Usamos la media de las representaciones para obtener el embedding de la oración
    sentence_embedding = word_embeddings.mean(dim=0)

    # Se calcula la similitud entre las palabras y la oración
    similarities = torch.nn.functional.cosine_similarity(word_embeddings, sentence_embedding.unsqueeze(0), dim=1)

    # Ordenamos por similitud y obtener los índices de las palabras más similares
    sorted_indices = torch.argsort(similarities, descending=True).tolist()
    
    # Convertir tokens a palabras originales
    tokens_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'].squeeze(0).tolist())

    # Obtenemos las top_n palabras más representativas
    top_palabras = [tokens_text[idx] for idx in sorted_indices if tokens_text[idx] not in ["[CLS]", "[SEP]"]][:top_n]

    return top_palabras

# Función para calcular la similitud entre documentos usando BERT
def bert_similarity(doc1, doc2, model, tokenizer):
    tokens1 = tokenizer(doc1, return_tensors="pt", truncation=True, padding=True)
    tokens2 = tokenizer(doc2, return_tensors="pt", truncation=True, padding=True)
    
    with torch.no_grad():
        output1 = model(**tokens1)
        output2 = model(**tokens2)
    
    embedding1 = output1.last_hidden_state.mean(dim=1).squeeze()
    embedding2 = output2.last_hidden_state.mean(dim=1).squeeze()
    
    cosine_similarity = torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=0)
    return cosine_similarity.item()



1. Generación de cuerpo de documentos

In [11]:
# Librerías utilizadas: nltk, PlaintextCorpusReader
# Descripción: Lee y organiza documentos de texto para su posterior análisis

# Definimos el lugar donde se encuentran los archivos de texto
corpus_root = ''
corpus_files = [f'Libro_{i}.txt' for i in range(1, 11)]

# Creamos el corpus con la codificación utf8
corpus = PlaintextCorpusReader(corpus_root, corpus_files, encoding='utf-8')


2. Preparación de texto

In [12]:
# Librerías utilizadas: nltk, re, stopwords
# Funciones utilizadas: clean_text, aplicar_procesamiento, pos_tag_text
# Descripción: Limpia, preprocesa y etiqueta gramaticalmente el texto de los documentos

# Procesar y preprocesar cada documento en el corpus
tagged_docs = {}
for fileid in corpus.fileids():
    raw_text = corpus.raw(fileid)
    cleaned_text = clean_text(raw_text)
    
    # Aplicar preprocesamiento
    cuerpos_procesados = aplicar_procesamiento([cleaned_text])
    cuerpos_procesados = ' '.join(cuerpos_procesados[0])
    
    tagged_docs[fileid] = pos_tag_text(cuerpos_procesados)


3. Similitud de palabras con synsets

In [13]:
# Librerías utilizadas: nltk, wordnet
# Funciones utilizadas: most_frequent_word, word_similarity
# Descripción: Encuentra la palabra más frecuente en cada categoría gramatical y calcula sus similitudes con otras palabras usando synsets

# Encontrar las palabras más frecuentes y calcular sus similitudes
for fileid, tagged_text in tagged_docs.items():
    flat_tagged_text = [item for sublist in tagged_text for item in sublist]
    
    # Verbo más frecuente
    most_freq_verb = most_frequent_word(flat_tagged_text, 'VB')
    verb_similarities_wup = word_similarity(most_freq_verb, wn.VERB, metric='wup_similarity')
    verb_similarities_path = word_similarity(most_freq_verb, wn.VERB, metric='path_similarity')
    
    # Sustantivo más frecuente
    most_freq_noun = most_frequent_word(flat_tagged_text, 'NN')
    noun_similarities_wup = word_similarity(most_freq_noun, wn.NOUN, metric='wup_similarity')
    noun_similarities_path = word_similarity(most_freq_noun, wn.NOUN, metric='path_similarity')
    
    print(f"Documento: {fileid}")
    print(f"Verbo más frecuente: {most_freq_verb}")
    print("Similitudes WUP (verbo):", verb_similarities_wup)
    print("Similitudes Path (verbo):", verb_similarities_path)
    print(f"Sustantivo más frecuente: {most_freq_noun}")
    print("Similitudes WUP (sustantivo):", noun_similarities_wup)
    print("Similitudes Path (sustantivo):", noun_similarities_path)
    print("\n")


Documento: Libro_1.txt
Verbo más frecuente: said
Similitudes WUP (verbo): [(Synset('say.v.11'), 0.5333333333333333), (Synset('say.v.07'), 0.5333333333333333), (Synset('say.v.09'), 0.5), (Synset('pronounce.v.01'), 0.5), (Synset('pronounce.v.01'), 0.4)]
Similitudes Path (verbo): [(Synset('say.v.08'), 0.3333333333333333), (Synset('say.v.09'), 0.3333333333333333), (Synset('state.v.01'), 0.3333333333333333), (Synset('pronounce.v.01'), 0.3333333333333333), (Synset('pronounce.v.01'), 0.25)]
Sustantivo más frecuente: rnine
Similitudes WUP (sustantivo): []
Similitudes Path (sustantivo): []


Documento: Libro_2.txt
Verbo más frecuente: said
Similitudes WUP (verbo): [(Synset('say.v.11'), 0.5333333333333333), (Synset('say.v.07'), 0.5333333333333333), (Synset('say.v.09'), 0.5), (Synset('pronounce.v.01'), 0.5), (Synset('pronounce.v.01'), 0.4)]
Similitudes Path (verbo): [(Synset('say.v.08'), 0.3333333333333333), (Synset('say.v.09'), 0.3333333333333333), (Synset('state.v.01'), 0.3333333333333333), (Sy

4. Similitud de documentos con synsets

In [14]:
# Librerías utilizadas: nltk, wordnet
# Funciones utilizadas: doc_to_synsets, document_path_similarity
# Descripción: Extrae la frase más representativa de cada documento y calcula la similitud entre documentos usando synsets

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
# Extraer la frase más representativa y calcular similitud entre documentos
frases_representativas = {}
for fileid in corpus.fileids():
    raw_text = corpus.raw(fileid)
    cleaned_text = clean_text(raw_text)
    
    sentences = nltk.sent_tokenize(cleaned_text)
    primer_capitulo = ' '.join(sentences)
    top_palabras = obtener_top_palabras(primer_capitulo, model, tokenizer)
    frases_representativas[fileid] = ' '.join(top_palabras)

# Comparar las frases más representativas utilizando la métrica "Path_similarity"
comparacion_similitud = []
libros = list(frases_representativas.keys())
for i in range(len(libros)):
    for j in range(i + 1, len(libros)):
        doc1 = frases_representativas[libros[i]]
        doc2 = frases_representativas[libros[j]]
        similarity = document_path_similarity(doc1, doc2)
        comparacion_similitud.append((libros[i], libros[j], similarity))

# Mostrar los resultados
for libro1, libro2, similarity in comparacion_similitud:
    print(f"Similitud entre {libro1} y {libro2}: {similarity:.2f}")


Similitud entre Libro_1.txt y Libro_2.txt: 0.78
Similitud entre Libro_1.txt y Libro_3.txt: 0.77
Similitud entre Libro_1.txt y Libro_4.txt: 0.11
Similitud entre Libro_1.txt y Libro_5.txt: 0.66
Similitud entre Libro_1.txt y Libro_6.txt: 0.66
Similitud entre Libro_1.txt y Libro_7.txt: 0.14
Similitud entre Libro_1.txt y Libro_8.txt: 0.10
Similitud entre Libro_1.txt y Libro_9.txt: 0.12
Similitud entre Libro_1.txt y Libro_10.txt: 0.85
Similitud entre Libro_2.txt y Libro_3.txt: 0.55
Similitud entre Libro_2.txt y Libro_4.txt: 0.11
Similitud entre Libro_2.txt y Libro_5.txt: 0.62
Similitud entre Libro_2.txt y Libro_6.txt: 0.44
Similitud entre Libro_2.txt y Libro_7.txt: 0.14
Similitud entre Libro_2.txt y Libro_8.txt: 0.10
Similitud entre Libro_2.txt y Libro_9.txt: 0.12
Similitud entre Libro_2.txt y Libro_10.txt: 0.63
Similitud entre Libro_3.txt y Libro_4.txt: 0.10
Similitud entre Libro_3.txt y Libro_5.txt: 0.44
Similitud entre Libro_3.txt y Libro_6.txt: 0.44
Similitud entre Libro_3.txt y Libro_7.

5. Similitud de palabras con embedding (GloVe)

In [15]:
# Librerías utilizadas: numpy, scipy
# Funciones utilizadas: load_glove_embeddings, find_closest_embeddings
# Descripción: Encuentra las palabras más similares al verbo más frecuente usando embeddings de GloVe

# Cargar las incrustaciones de GloVe
embeddings_dict = load_glove_embeddings('glove.6B.50d.txt')

# Encontrar los términos más similares al verbo más frecuente en cada documento
for fileid, tagged_text in tagged_docs.items():
    flat_tagged_text = [item for sublist in tagged_text for item in sublist]
    
    # Verbo más frecuente
    most_freq_verb = most_frequent_word(flat_tagged_text, 'VB')
    
    # Embedding del verbo más frecuente
    if most_freq_verb in embeddings_dict:
        most_freq_verb_embedding = embeddings_dict[most_freq_verb]
        similar_terms = find_closest_embeddings(most_freq_verb_embedding, embeddings_dict)
        print(f"Documento: {fileid}")
        print(f"Verbo más frecuente: {most_freq_verb}")
        print("Términos más similares:", similar_terms)
        print("\n")


Documento: Libro_1.txt
Verbo más frecuente: said
Términos más similares: ['said', 'told', 'says', 'spokesman', 'saying']


Documento: Libro_2.txt
Verbo más frecuente: said
Términos más similares: ['said', 'told', 'says', 'spokesman', 'saying']


Documento: Libro_3.txt
Verbo más frecuente: came
Términos más similares: ['came', 'took', 'after', 'saw', 'when']


Documento: Libro_4.txt
Verbo más frecuente: said
Términos más similares: ['said', 'told', 'says', 'spokesman', 'saying']


Documento: Libro_5.txt
Verbo más frecuente: said
Términos más similares: ['said', 'told', 'says', 'spokesman', 'saying']


Documento: Libro_6.txt
Verbo más frecuente: said
Términos más similares: ['said', 'told', 'says', 'spokesman', 'saying']


Documento: Libro_7.txt
Verbo más frecuente: said
Términos más similares: ['said', 'told', 'says', 'spokesman', 'saying']


Documento: Libro_8.txt
Verbo más frecuente: fleming
Términos más similares: ['fleming', 'stuart', 'clarke', 'healy', 'fletcher']


Documento: Libr

6. Similitud de documentos con embedding (BERT)

In [16]:
# Librerías utilizadas: torch, transformers
# Funciones utilizadas: bert_similarity
# Descripción: Calcula la similitud entre documentos usando embeddings de BERT

# Cargar el modelo BERT pre-entrenado y el tokenizador (largo y uncased)
bert_tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
bert_model = BertModel.from_pretrained("bert-large-uncased")

# Comparar las frases más representativas utilizando BERT
comparacion_similitud_bert = []
for i in range(len(libros)):
    for j in range(i + 1, len(libros)):
        doc1 = frases_representativas[libros[i]]
        doc2 = frases_representativas[libros[j]]
        similarity = bert_similarity(doc1, doc2, bert_model, bert_tokenizer)
        comparacion_similitud_bert.append((libros[i], libros[j], similarity))

# Mostrar los resultados de BERT
for libro1, libro2, similarity in comparacion_similitud_bert:
    print(f"Similitud BERT entre {libro1} y {libro2}: {similarity:.2f}")


Similitud BERT entre Libro_1.txt y Libro_2.txt: 0.62
Similitud BERT entre Libro_1.txt y Libro_3.txt: 0.65
Similitud BERT entre Libro_1.txt y Libro_4.txt: 0.67
Similitud BERT entre Libro_1.txt y Libro_5.txt: 0.65
Similitud BERT entre Libro_1.txt y Libro_6.txt: 0.61
Similitud BERT entre Libro_1.txt y Libro_7.txt: 0.68
Similitud BERT entre Libro_1.txt y Libro_8.txt: 0.59
Similitud BERT entre Libro_1.txt y Libro_9.txt: 0.69
Similitud BERT entre Libro_1.txt y Libro_10.txt: 0.65
Similitud BERT entre Libro_2.txt y Libro_3.txt: 0.89
Similitud BERT entre Libro_2.txt y Libro_4.txt: 0.64
Similitud BERT entre Libro_2.txt y Libro_5.txt: 0.67
Similitud BERT entre Libro_2.txt y Libro_6.txt: 0.93
Similitud BERT entre Libro_2.txt y Libro_7.txt: 0.63
Similitud BERT entre Libro_2.txt y Libro_8.txt: 0.64
Similitud BERT entre Libro_2.txt y Libro_9.txt: 0.69
Similitud BERT entre Libro_2.txt y Libro_10.txt: 0.91
Similitud BERT entre Libro_3.txt y Libro_4.txt: 0.66
Similitud BERT entre Libro_3.txt y Libro_5.t